<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/representaciones_latentes/autocodificador_variacional_vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción

Este *notebook* muestra como construir un autocodificador variacional, mejor conocido como Variational Autoencoder (VAE) entrenando un modelo en el conjunto de datos MNIST para generar nuevas imágenes.

Un autocodificador variacional es un modelo generativo basado en probabilidad. Consiste en un codificador, que toma datos x como entrada y los transforma en una representación latente z, y un decodificador, que toma una representación latente z y devuelve una reconstrucción y. La inferencia se realiza a través de la inferencia variacional para aproximar la parte posterior del modelo.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from IPython import display

In [2]:
batch_size = 128
latent_dim = 2

In [5]:
def map_image(image, label):
    image = tf.cast(image, dtype=tf.float32)
    image /= 255.0
    image = tf.reshape(image, shape=(28,28,1,))
    return image

def get_dataset(map_fn, is_validation=False):
    if is_validation:
        split_name = "test"
    else:
        split_name = "train"
  
    dataset = tfds.load("mnist", as_supervised=True, split=split_name)
    dataset = dataset.map(map_fn)

    if is_validation:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.shuffle(1024).batch(batch_size)
  
    return dataset

In [6]:
train_dataset = get_dataset(map_image)

In [7]:
class Sampling(tf.keras.layers.Layer):
  def call(self, inputs):
    mu, sigma = inputs

    batch = tf.shape(mu)[0]
    dim = tf.shape(mu)[1]

    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))

    return mu + tf.exp(0.5 * sigma) * epsilon

In [8]:
def encoder_layers(inputs, latent_dim):
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding="same", activation="relu", name="encode_conv1")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding="same", activation="relu", name="encoder_conv2")(x)

    batch_2 = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Flatten(name="encode_flatten")(batch_2)

    x = tf.keras.layers.Dense(20, activation="relu", name="encoder_dense")(x)
    x = tf.keras.layers.BatchNormalization()(x)

    mu = tf.keras.layers.Dense(latent_dim, name="latent_mu")(x)
    sigma = tf.keras.layers.Dense(latent_dim, name="latent_sigma")(x)

    return mu, sigma, batch_2.shape

In [9]:
def encoder_model(latent_dim, input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)

    mu, sigma, conv_shape = encoder_layers(inputs, latent_dim=latent_dim)
    z = Sampling()((mu, sigma))
    model = tf.keras.Model(inputs=inputs, outputs=[mu, sigma, z])

    return model, conv_shape

**Por hacer**

* Descripción del modelo VAE
* Decodificador
* Función de costo
* Entrenamiento de modelo
* Predicciones
* Comentar funciones
* Referencia
